In [1]:
from manim import *
from common import nuclear
from importlib import reload
import random

Manim Community v0.17.3

In [59]:
%%manim -v WARNING --disable_caching -qm MainScene

reload(nuclear)

config.media_width = "75%"
config.media_embed = True

PARTICLE_SIZE_MULTIPLIER = 0.3

class MainScene(Scene):
    def construct(self):
        nucleus = nuclear.Nucleus().init_from_nums(12, 12, 0.5, 4, PARTICLE_SIZE_MULTIPLIER, seed=6)
        self.wait(1)
        # Alpha decay
        self.play(*nucleus.create_anims())  # show the nucleus on screen
        daughter1, daughter2, pairs1, pairs2 = nucleus.decay(2, 2, (2, -1), False, False, 1)

        daughter1.shift(2 * RIGHT + DOWN)
        
        self.play(*map(lambda p: ReplacementTransform(p[0], p[1]), pairs1 + pairs2))

        self.play(daughter2.animate.shift(2 * LEFT))
        
        # Beta + decay
        # pick a nucleon to decay
        proton = daughter2.find_closest_nucleon((-2, -1), nuclear.PROTON)
        # Turn it into a neutron
        neutron = nuclear.Particle(nuclear.NEUTRON, PARTICLE_SIZE_MULTIPLIER)
        neutron.move_to(proton)

        electron = nuclear.Particle(nuclear.POSITRON, PARTICLE_SIZE_MULTIPLIER)
        electron.move_to(proton)

        self.add(electron)
        self.play(Transform(proton, neutron), electron.animate.shift(2 * LEFT + DOWN), run_time=1)
        
        # Gamma decay
        new_daughter2_nucleons = daughter2.get_nucleons_list()
        random.shuffle(new_daughter2_nucleons)
        new_daughter2 = nuclear.Nucleus().init_from_nucleons(new_daughter2_nucleons, 0.5, 4, PARTICLE_SIZE_MULTIPLIER, False)
        new_daughter2.move_to(daughter2)
        # Find some point on the edge of the nucleus
        gamma_init_coords = daughter2.find_closest_nucleon((0, 0)).get_center()
        initial_arrow = Arrow(start=gamma_init_coords, end=gamma_init_coords)
        final_arrow = Arrow(start=gamma_init_coords, end=(4 * RIGHT + 2 * UP))
        self.add(initial_arrow)
        # Go from each original nucleon to the shuffled one
        self.play(ReplacementTransform(initial_arrow, final_arrow), *map(lambda n: ReplacementTransform(n[0], n[1]), zip(new_daughter2_nucleons, new_daughter2.get_nucleons_list())))
        self.wait(1)